## RAG pipeline

In [ ]:
!pip install -qU pydantic_ai
!pip install -qU langchain langchain-community langchain-huggingface faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.37.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.3 MB/s eta 0:00:00
   ━━━━

In [ ]:
import os
import getpass
import nest_asyncio #https://ai.pydantic.dev/troubleshooting/
nest_asyncio.apply()

os.environ["GOOGLE_KEY"] = getpass.getpass("Google AI Studio API key :")

Google AI Studio API key :··········


In [ ]:
# IMPORTS
from pydantic import BaseModel
from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.models.gemini import GeminiModel

from typing import List
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
# EXTRACT VECTORDB FILES
#!unzip -u /content/vectorDB_mini.zip

Archive:  /content/vectorDB_mini.zip
   creating: vectorDB/
  inflating: vectorDB/index.faiss    
  inflating: vectorDB/index.pkl      


In [48]:
# MODEL DEFINITION
model = GeminiModel(
    'gemini-2.0-flash-exp',
    api_key=os.getenv('GOOGLE_KEY')
)

# SYSTEM PROMPT (STATIC)
system_prompt = """
Tu es un assitant pour la solution SAGE X3 ERP uniquement.

Tu connais le language 4GL, car celui-ci est le language de programmation utilisé dans SAGE X3 ERP.

Ta tâche consiste à répondre à la demande de l'utilisateur, avec des exemples de code en 4GL si nécessaire, toujours dans le contexte de SAGE X3 ERP.
First, use the 'get_sagex3_documentation' tool to get specific information on SAGE X3 ERP.

Explique toujours ta réponse en détaillant chacune des étapes de ta réflexion.

Si tu ne connais pas la réponse, dis-le à l'utilisateur - n'essaie pas de l'inventer !
"""

# DEPENDANCES
embedding_model = HuggingFaceEmbeddings(model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

class Deps(BaseModel):
    retriever: VectorStoreRetriever

deps=Deps(
    retriever=FAISS.load_local('/content/vectorDB', embedding_model, allow_dangerous_deserialization=True).as_retriever(search_type="mmr", search_kwargs={"k": 10})
)

# INSTANCIATE AGENT WITH VECTORSTORE
agent = Agent(
    model,
    system_prompt=system_prompt,
    deps_type=Deps,
    retries=2
)

In [49]:
# RAG TOOL
@agent.tool
def get_sagex3_documentation(ctx: RunContext[Deps], query: str) -> List[str]:
    """Rechercher dans la documentation SAGE X3 ERP.

    Args:
        ctx: le context.
        query: la requête pour rechercher dans la documentation, avec les principaux mots-clés.

    Returns:
        List[str]: les résultats de la recherche.
    """
    print(f"Using get_sagex3_documentation() with query: {query}.\n")

    docs = ctx.deps.retriever.invoke(query)

    print("Docs retrieved from vectorstore :\n")
    for doc in docs:
      print(doc)
      print("\n")
    print("-"* 50 + "\n\n")

    return [doc.page_content for doc in docs]


In [50]:
# RUN AGENT
result = agent.run_sync('comment ajouter une nouvelle société', deps=deps)
print(result.data)

Using get_sagex3_documentation() with query: création société.
Using get_sagex3_documentation() with query: ajout société.


Docs retrieved from vectorstore :

page_content=' '
Docs retrieved from vectorstore :

page_content=' '


page_content='La gestion des variables dans les blocs tableaux de Sage X3, combinée avec les techniques de Trans-Classe et d'alimentation champ par champ, permet aux développeurs de construire des interfaces utilisateur dynamiques et efficaces, adaptées aux besoins spécifiques des entreprises. Cette approche offre un équilibre entre automatisation et contrôle personnalisé, essentiel pour le traitement précis et efficace des données dans les applications ERP.' metadata={'h1': 'Mise à jour de la variable de bas de tableau', 'h3': 'Conclusion', 'source': 'Sage X3 L4G.md'}


page_content='Utilisation : Réactive les zones spécifiées, les rendant à nouveau saisissables.  
Exemple :  
«Actzo [M:SOH0] SOHNUM»' metadata={'h1': 'Manipulation des Écrans dans Sage X3', '

## GRADIO

In [35]:
!pip install -qU gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00


In [40]:
import gradio as gr

In [ ]:

def agent_invoke(query) -> str:
  history = []
  result = agent.run_sync(query, message_history=history, deps=deps)
  history = result.all_messages()
  return result.data

demo = gr.Interface(
    fn=agent_invoke,
    inputs=["text"],
    outputs=["text"],
)
demo.launch(debug=True)

# gr.ChatInterface(
#     agent_invoke,
#     chatbot=gr.Chatbot(height=300),
#     textbox=gr.Textbox(placeholder="Pose moi des questions sur SAGE X3 ERP", container=False, scale=7),
#     title="SAGE X3 ERP assistant",
#     description="SAGE X3 ERP assistant",
#     theme="ocean"
# ).launch(debug=True)



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://56eec8fcce422f1a1c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
MACWARREQ  MWRMWR0MACNUMYes
MAILING  OMMOMM0OMMNUMNo
MAILING  OMMOMM1CMGNUMYes
MAILING  OMMOMM2CREDATYes
MAILXML  MXLMXL0NUMNo
MAILXMLD  MXDMXD0NUM+TAB+FIENo
MANDATE  MDTMDT0CPY+UMRNUMNo
MANDATE  MDTMDT1CPY+BPCNUM+UMRNUMNo
MARASSDEF  MADMAD0RECORDNUM+RECORDADD+RECORDTYPNo
MARASSDEF  MADMAD1RECORDNUM+RECORDTYPYes
MARASSREP  MARmars-00RECORDNUM+RECORDADD+RECORDTYPYes
MARASSREP  MAR01-marsRECORDNUM+RECORDTYPYes
MARASSREP  MAR02-marsRECORDNUM+RECORDADD+RECORDTYP+REPTYPYes
MARASSREP  MAR03-marsRECORDNUM+RECORDTYP+REPTYPYes
MARDEF  MDFMDF0MARSCTNUMYes
MARDEF  MDFMDF1MARSCTNUM+CRINUMNo
MARDEF  MDFMDF2MARSCTNUM+CRINUM+MARSCTTAB+MARSCTFIEYes
MARDEFVAL  MDVMDV0MARSCTNUM+CRINUMYes
MARDEFVAL  MDVMDV1MARSCTNUM+CRINUM+SCTVALNUMNo
MARREPSEC  MRSMRS0MARSCTNUMYes
MARREPSEC  MRSMRS1MARSCTNUM+SCTREPSECNo
MARSCT  MSTMST0MARSCTNUMNo
MATCHCODE  MTCMTC0COA+ACC+BPRNo
MATCHTOL  MATMAT0MATTOLNo
MATCSTW  MAWMAW0MAWUID+STOFCY+ITMREF+ECC